Summary of outcomes:

div to ssh_cos only, batch_size=10
R2 from the best models in each run are:
[0.71200164 0.7092568  0.70846646 0.71314044]
corr from the best models in each run are:
[0.8454739  0.84274485 0.84231521 0.8457262 ]

vort to ssh_cos only, batch_size=10
R2 from the best models in each run are:
[0.83241847 0.82840353 0.82703668 0.82710327]
corr from the best models in each run are:
[0.91324089 0.91147217 0.91132234 0.91095138]

---------------------------------------------
div to ssh_cos and ssh_sin, batch_size=10
R2 from the best models in each run are:
[0.77560043 0.771282   0.77819786 0.7697045 ]
corr from the best models in each run are:
[0.88150735 0.88002645 0.8826575  0.87888121]


vort to ssh_cos and ssh_sin, batch_size=10
R2 from the best models in each run are:
[0.74091076 0.73678359 0.73939649 0.73511447]
corr from the best models in each run are:
[0.86151784 0.85899898 0.86089328 0.85819646]

---------------------------------------------
div to ssh_cos only, batch_size=100
R2 from the best models in each run are:
[0.70388961 0.69786856 0.70412668 0.69500771]
corr from the best models in each run are:
[0.84168713 0.83839298 0.84008428 0.83767132]

vort to ssh_cos only, batch_size=100
R2 from the best models in each run are:
[0.7655253  0.76617748 0.76492614 0.77463616]
corr from the best models in each run are:
[0.87496319 0.87541521 0.87478422 0.88030991]

---------------------------------------------

div to ssh_cos and ssh_sin, batch_size=100
R2 from the best models in each run are:
[0.76925564 0.76784907 0.7624617  0.76859775]
corr from the best models in each run are:
[0.87765357 0.87739345 0.87562212 0.87729587]


vort to ssh_cos and ssh_sin, batch_size=100
R2 from the best models in each run are:
[0.67702224 0.67874523 0.67075058 0.67682131]
corr from the best models in each run are:
[0.82337388 0.82430789 0.81952929 0.82334258]

In [1]:
#Previously I got weird results because my perforamnces from the vort and div run are the results of 
#Jeff's vort and div runs swapped. My run with (vort+div) is also slightly worse than Jeff's. Now I 
#try if I could reconstruct Jeff's results if I make the batch number to be 10 and make the output channel 
#to be ssh_cos, just like what Jeff did. 
#If these still don't agree with Jeff's results, then it is probably the batchnorm layer that's causing problems. 
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import r2_score as R2
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score as r2
from copy import deepcopy
import utils
from unet import UNet_nobatchnorm
from scipy.stats import pearsonr
#JU's addtion to automate inputs and outputs
import helper_functions as hf
import os
def save_fn(var_input_list, var_output_list):
    var_input_join  = '_and_'.join(var_input_list)
    var_output_join = '_and_'.join(var_output_list)
    return '{}_to_{}'.format(var_input_join, var_output_join)
    
torch.cuda.set_device(1)
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print ('Running on ', device)

Running on  cuda:1


In [2]:
torch.cuda.is_available()

True

In [3]:
print(torch.__version__)
print(torch.version.cuda)

2.5.1
12.6


In [4]:
maxEpochs =  300#small number is taken for debugging
nensemble = 4 #How many training sessions are run for each configuration ***
Nbase = 16

In [5]:
!nvidia-smi #GPU usage should be maxed out during training; tune batch_size according to that

Wed Apr  2 21:13:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:03:00.0 Off |                    0 |
| N/A   58C    P0            248W /  500W |   22493MiB /  81920MiB |     95%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [6]:
root_dir = '/work/uo0780/u241359/project_tide_synergy/data/'
nctrains, nctest = hf.load_data_from_nc_as_lists(root_dir)

Ntrain = np.sum([nc.dimensions['time_counter'].size for nc in nctrains], axis = 0)
Ntest = np.sum([nc.dimensions['time_counter'].size for nc in nctest], axis = 0)

print('number of training records:', Ntrain)
print('number of testing records:', Ntest)

numTrainFiles = len(nctrains)
numRecsFile = nctrains[0].dimensions['time_counter'].size #How many snapshots in time in each data set there is
print (numRecsFile)

number of training records: 600
number of testing records: 150
150


In [7]:
def hwvorticity(u, v, dgrid = 4000):
    return (np.gradient(v, axis =2) - np.gradient(u, axis =1))/dgrid

def hwdivergence(u, v, dgrid = 4000):
    return (np.gradient(u, axis =2) + np.gradient(v, axis =1))/dgrid

def preload_data_vortdiv(nctrains, total_records):
    #total_records = Ntrain#sum(nc.dimensions['time_counter'].size for nc in nctrains)
    #dimensions of data of the nc file.
    max_height = 722
    max_width = 258
    all_input_data = np.zeros((total_records, N_inp, max_height, max_width))*np.nan
    all_output_data = np.zeros((total_records, N_out, max_height, max_width))*np.nan
    current_index = 0
    for ncindex, ncdata in enumerate(nctrains):
        num_recs = ncdata.dimensions['time_counter'].size #how many time stamps are there in each .nc file (i.e., at each turbulence level)
        rec_slice = slice(current_index, current_index + num_recs)
        for ind, var_name in enumerate(var_input_names):
            if var_name == 'vort':
                u = np.squeeze(ncdata.variables['u_xy_ins'])
                v = np.squeeze(ncdata.variables['v_xy_ins'])
                #u.shape: (150, 722, 257); v.shape: (150, 721, 258)
                #as u and v have different number of grid points in x and y, we truncate them so that their shapes agree, enabling the simple way to compute vorticities based on finite diff.
                data_slice = hwvorticity(u[:,:-1,:], v[:,:,:-1])
            elif var_name == 'div':
                u = np.squeeze(ncdata.variables['u_xy_ins'])
                v = np.squeeze(ncdata.variables['v_xy_ins'])
                data_slice = hwdivergence(u[:,:-1,:], v[:,:,:-1])
            else:           
                data_slice = np.squeeze(ncdata.variables[var_name])
            # print('data_slice shape:')
            # print(data_slice.shape)        
            #all_input_data[rec_slice, ind, :, :] = data_slice
            
            #For some variables, the dimensions in (x, y) may be smaller than (max_height, max_width). Changing the code so that it adapts them.
            # Get the actual dimensions of data_slice
            slice_height, slice_width = data_slice.shape[-2], data_slice.shape[-1]
            # Place data_slice into the corresponding slice of all_input_data
            all_input_data[rec_slice, ind, :slice_height, :slice_width] = data_slice
    

        for ind, var_name in enumerate(var_output_names):
            data_slice = np.squeeze(ncdata.variables[var_name])
            #all_output_data[rec_slice, ind, :, :] = data_slice
            # Get the actual dimensions of data_slice
            slice_height, slice_width = data_slice.shape[-2], data_slice.shape[-1]
            # Place data_slice into the corresponding slice of all_input_data
            all_output_data[rec_slice, ind, :slice_height, :slice_width] = data_slice

        current_index += num_recs
        
    return all_input_data, all_output_data
    


# Modify the loadtrain function to pull data from preloaded memory
def loaddata_preloaded_train(index, batch_size, all_input_data, all_output_data):
    rec_slice = slice(index, index + batch_size)
    lim = 720
    width = 256
    yslice = slice(0, lim)
    xslice = slice(0, width)
    # print('mean of squared values of loaded input data:')
    # print("{0:0.32f}".format(np.nanmean(all_input_data[rec_slice, :, yslice, xslice]**2)))
    return (all_input_data[rec_slice, :, yslice, xslice], 
            all_output_data[rec_slice, :, yslice, xslice])
#Load test data as one single batch
def loaddata_preloaded_test(all_input_data, all_output_data):
    #rec_slice = slice(index, index + batch_size)
    lim = 720
    width = 256
    yslice = slice(0, lim)
    xslice = slice(0, width)
    # print('rec_slice is:')
    # print(rec_slice)
    # print('mean of squared values of loaded input data:')
    # print("{0:0.32f}".format(np.nanmean(all_input_data[rec_slice, :, yslice, xslice]**2)))
    return (all_input_data[:, :, yslice, xslice], 
            all_output_data[:, :, yslice, xslice])



In [8]:
def run_model(var_input_names, var_output_names, save_fn_prefix, N_inp, N_out, iensemble, R2_all, corr_all):
    ytest_slice = slice(0, 720)
    xtest_slice = slice(0, 256)
    rectest_slice = slice(0, 150)

    def totorch(x):
        return torch.tensor(x, dtype = torch.float).cuda()

    model = UNet_nobatchnorm(N_inp, N_out, bilinear = True, Nbase = Nbase).cuda()
    #model = torch.compile(UNet(N_inp, N_out, bilinear = True, Nbase = Nbase).cuda())

    if iensemble == 0:
        input = torch.randn(1,N_inp,256,720).to(device) 
        output = model(input)
        print('Model has ', utils.nparams(model)/1e6, ' million params')

    # for index in range(0, Ntrain, batch_size):
    #     inp, out = loadtrain_preloaded(index, batch_size, all_train_input, all_train_output)
    #     print(inp.shape, out.shape)
#         print(np.nanmean(inp**2), np.max(inp**2), inp.shape)
#         print(np.nanmean(out**2), np.max(out**2), inp.shape)

    inp_test, out_test = loaddata_preloaded_test(all_test_input, all_test_output)
    #inp, out_test = loadtest()
    # print('shapes of input and output TEST data:')
    # print(inp_test.shape, out_test.shape)
    with torch.no_grad():
        inp_test = totorch(inp_test)

    Tcycle = 10
    criterion_train  = nn.L1Loss()
    optim = torch.optim.AdamW(model.parameters(), lr=lr0, betas=(0.9, 0.999), eps=1e-8, weight_decay=1e-5*100) #increase weight_decay ***

    r2_test = np.zeros(maxEpochs)
    epochmin = []
    maxr2l = []

    learn = np.zeros(maxEpochs)
    minloss = 1000
    maxR2 = -1000
    minlosscount = 0
    perm = False

    model_best = deepcopy(model)  # Initialize before the loop for safety

    #print('Starting training loop')
    for epoch in tqdm(range(maxEpochs)):
        lr = utils.cosineSGDR(optim, epoch, T0=Tcycle, eta_min=0, eta_max=lr0, scheme = 'constant')  #captioning this seems to make the printed corr larger??***
        model.train()
        index_perm = np.arange(0, Ntrain, batch_size)
        
        if perm:
            index_perm = np.random.permutation(index_perm)
        
        for index in index_perm:
            inp, out = loaddata_preloaded_train(index, batch_size, all_train_input, all_train_output)            
#           inp, out = loadtrain(index, batch_size, nctrains)
            inp, out = totorch(inp), totorch(out)
            #continue #do this to pause the later operations to check how long it takes for the steps up to this 
            out_mod = model(inp)
            loss = criterion_train(out.squeeze(), out_mod.squeeze())
            #Set gradient to zero
            optim.zero_grad()
            #Compute gradients       
            loss.backward()
            #Update parameters with new gradient
            optim.step()
            #Record train loss
            #scheduler.step()
          
        model.eval()
        with torch.no_grad():
            #model_cpu = model.to('cpu')
            #out_mod = model_cpu(inp_test.to('cpu'))
            out_mod=model(inp_test)
            
            r2 = R2(out_test.flatten(), (out_mod).cpu().numpy().flatten())
            r2_test[epoch] = r2
            #print('Debugging: R2 of current epoch:', r2)#Debugging
            #record current best model and best predictions
            if maxR2 <  r2:
                maxR2 = r2
                epochmin.append(epoch)
                maxr2l.append(maxR2)                
                model_best = deepcopy(model)
                corr, pval = pearsonr(out_test.flatten(), (out_mod).cpu().numpy().flatten())
                print('R2:', r2, ' corr: ', corr, ' pval: ', pval)
            #model = model_cpu.to(device)

    #_, out_test = loadtest()
    model_best.eval()
    with torch.no_grad():
    #     inp_test = totorch(inp)
        model_best.to('cpu') #added by HW 
        out_mod = model_best(inp_test.to('cpu')).detach().cpu().numpy()

    R2_all[iensemble]=R2(out_test.flatten(), out_mod.flatten())
    corr_all[iensemble]=pearsonr(out_test.flatten(), out_mod.flatten())[0]
    # print('Best model R2:', R2_all[iensemble])#pearsonr(out_test.flatten(), out_mod.flatten())[0])
    # print('Best model corr:', corr_all[iensemble])#R2(out_test.flatten(), out_mod.flatten()))

    # Nx, Ny = out_test.shape[2:]; Nx, Ny

    # print(out_mod.shape, 'outout model shape')
    dr = '/work/uo0780/u241359/project_tide_synergy/trainedmodels' #'./models/to_vel'
    os.makedirs(dr, exist_ok=True) # exist_ok=True allows the function to do nothing (i.e., not raise an error) if the directory already exists.
    fstr = f'{save_fn_prefix}_rp_{iensemble}'
    PATH = dr + f'/{fstr}.pth'
    torch.save(model_best.state_dict(), PATH)

In [9]:
vi1 = 'div'

vo1 = 'ssh_cos'

save_fn_prefix  = 'any_{}_{}_smallbatch_cosonly_nobatchnorm'.format(vi1, vo1)
var_input_names = [vi1]
var_output_names = [vo1]

batch_size = 10 

N_inp = len(var_input_names)
N_out = len(var_output_names)

lr0 = 0.005*10/batch_size #Roughly should scale inversely to batch_size

#Recording performance metrics on test data after eaching training cycle
R2_all = np.zeros(nensemble)
corr_all = np.zeros(nensemble)

nctrains, nctest = hf.load_data_from_nc_as_lists(root_dir)

all_train_input, all_train_output = preload_data_vortdiv(nctrains, Ntrain)
all_test_input, all_test_output = preload_data_vortdiv(nctest, Ntest)

#Normalize data
#Compute mean and variance for normalization
mean_input=np.nanmean(np.concatenate((all_train_input, all_test_input), axis=0),axis=(0, 2, 3))
mean_output=np.nanmean(np.concatenate((all_train_output, all_test_output), axis=0),axis=(0, 2, 3))
#Subtract the data with their means
all_train_input=all_train_input-mean_input[None, :, None, None]
all_train_output=all_train_output-mean_output[None, :, None, None]
all_test_input=all_test_input-mean_input[None, :, None, None]
all_test_output=all_test_output-mean_output[None, :, None, None]
#Compute the variances
var_input=np.nanmean((np.concatenate((all_train_input, all_test_input), axis=0))**2,axis=(0, 2, 3))
var_output=np.nanmean((np.concatenate((all_train_output, all_test_output), axis=0))**2,axis=(0, 2, 3))
print("mean and variance of all input data:")
print(mean_input,var_input)
print("mean and variance of all output data:")
print(mean_output,var_output)
#Scale the data so that they have variance of 1
all_train_input=all_train_input/np.sqrt(var_input[None, :, None, None])
all_train_output=all_train_output/np.sqrt(var_output[None, :, None, None])
all_test_input=all_test_input/np.sqrt(var_input[None, :, None, None])
all_test_output=all_test_output/np.sqrt(var_output[None, :, None, None])
#Have checked that after these operations, the data is scaled to be zero mean and variance 1.

for iensemble in np.arange(nensemble):
    run_model(var_input_names, var_output_names, save_fn_prefix, N_inp, N_out, iensemble, R2_all, corr_all)  
print('R2 from the best models in each run are:')
print(R2_all)
print('corr from the best models in each run are:')
print(corr_all)

mean and variance of all input data:
[3.2595452e-09] [3.62891008e-11]
mean and variance of all output data:
[-0.00051623] [9.36516511e-05]
Model has  1.124401  million params


  0%|          | 1/300 [00:06<30:15,  6.07s/it]

R2: 0.4663811463231238  corr:  0.7132612632150401  pval:  0.0


  1%|          | 2/300 [00:11<29:12,  5.88s/it]

R2: 0.49390654730086836  corr:  0.7494709256339285  pval:  0.0


...

R2: 0.7121017565722534  corr:  0.8446821939041775  pval:  0.0


 13%|█▎        | 39/300 [03:35<24:49,  5.71s/it]

R2: 0.7131391989957544  corr:  0.8457262348076483  pval:  0.0


100%|██████████| 300/300 [25:04<00:00,  5.01s/it]


R2 from the best models in each run are:
[0.71200164 0.7092568  0.70846646 0.71314044]
corr from the best models in each run are:
[0.8454739  0.84274485 0.84231521 0.8457262 ]


In [10]:
vi1 = 'div'

vo1 = 'ssh_cos'
vo2 = 'ssh_sin'

save_fn_prefix  = 'any_{}_{}{}_smallbatch_nobatchnorm'.format(vi1, vo1, vo2)
var_input_names = [vi1]
var_output_names = [vo1, vo2]

batch_size = 10 

N_inp = len(var_input_names)
N_out = len(var_output_names)

lr0 = 0.005*10/batch_size #Roughly should scale inversely to batch_size

#Recording performance metrics on test data after eaching training cycle
R2_all = np.zeros(nensemble)
corr_all = np.zeros(nensemble)

nctrains, nctest = hf.load_data_from_nc_as_lists(root_dir)

all_train_input, all_train_output = preload_data_vortdiv(nctrains, Ntrain)
all_test_input, all_test_output = preload_data_vortdiv(nctest, Ntest)

#Normalize data
#Compute mean and variance for normalization
mean_input=np.nanmean(np.concatenate((all_train_input, all_test_input), axis=0),axis=(0, 2, 3))
mean_output=np.nanmean(np.concatenate((all_train_output, all_test_output), axis=0),axis=(0, 2, 3))
#Subtract the data with their means
all_train_input=all_train_input-mean_input[None, :, None, None]
all_train_output=all_train_output-mean_output[None, :, None, None]
all_test_input=all_test_input-mean_input[None, :, None, None]
all_test_output=all_test_output-mean_output[None, :, None, None]
#Compute the variances
var_input=np.nanmean((np.concatenate((all_train_input, all_test_input), axis=0))**2,axis=(0, 2, 3))
var_output=np.nanmean((np.concatenate((all_train_output, all_test_output), axis=0))**2,axis=(0, 2, 3))
print("mean and variance of all input data:")
print(mean_input,var_input)
print("mean and variance of all output data:")
print(mean_output,var_output)
#Scale the data so that they have variance of 1
all_train_input=all_train_input/np.sqrt(var_input[None, :, None, None])
all_train_output=all_train_output/np.sqrt(var_output[None, :, None, None])
all_test_input=all_test_input/np.sqrt(var_input[None, :, None, None])
all_test_output=all_test_output/np.sqrt(var_output[None, :, None, None])
#Have checked that after these operations, the data is scaled to be zero mean and variance 1.

for iensemble in np.arange(nensemble):
    run_model(var_input_names, var_output_names, save_fn_prefix, N_inp, N_out, iensemble, R2_all, corr_all)  
print('R2 from the best models in each run are:')
print(R2_all)
print('corr from the best models in each run are:')
print(corr_all)

mean and variance of all input data:
[3.2595452e-09] [3.62891008e-11]
mean and variance of all output data:
[-5.16228102e-04 -9.83592627e-05] [9.36516511e-05 1.01456128e-04]
Model has  1.124418  million params


  0%|          | 1/300 [00:09<46:33,  9.34s/it]

R2: 0.039152575088112984  corr:  0.7135761386136017  pval:  0.0


  1%|          | 2/300 [00:18<46:47,  9.42s/it]

R2: 0.6448040315219005  corr:  0.8087175468879996  pval:  0.0


...

R2: 0.7690721391740467  corr:  0.8782538890648184  pval:  0.0


 17%|█▋        | 50/300 [05:43<28:51,  6.93s/it]

R2: 0.7697191687263948  corr:  0.878884545698211  pval:  0.0


100%|██████████| 300/300 [30:14<00:00,  6.05s/it]


R2 from the best models in each run are:
[0.77560043 0.771282   0.77819786 0.7697045 ]
corr from the best models in each run are:
[0.88150735 0.88002645 0.8826575  0.87888121]


In [11]:
vi1 = 'div'

vo1 = 'ssh_cos'
vo2 = 'ssh_sin'

save_fn_prefix  = 'any_{}_{}{}_nobatchnorm'.format(vi1, vo1, vo2)
var_input_names = [vi1]
var_output_names = [vo1, vo2]

batch_size = 100

N_inp = len(var_input_names)
N_out = len(var_output_names)

lr0 = 0.005*10/batch_size #Roughly should scale inversely to batch_size

#Recording performance metrics on test data after eaching training cycle
R2_all = np.zeros(nensemble)
corr_all = np.zeros(nensemble)

nctrains, nctest = hf.load_data_from_nc_as_lists(root_dir)

all_train_input, all_train_output = preload_data_vortdiv(nctrains, Ntrain)
all_test_input, all_test_output = preload_data_vortdiv(nctest, Ntest)

#Normalize data
#Compute mean and variance for normalization
mean_input=np.nanmean(np.concatenate((all_train_input, all_test_input), axis=0),axis=(0, 2, 3))
mean_output=np.nanmean(np.concatenate((all_train_output, all_test_output), axis=0),axis=(0, 2, 3))
#Subtract the data with their means
all_train_input=all_train_input-mean_input[None, :, None, None]
all_train_output=all_train_output-mean_output[None, :, None, None]
all_test_input=all_test_input-mean_input[None, :, None, None]
all_test_output=all_test_output-mean_output[None, :, None, None]
#Compute the variances
var_input=np.nanmean((np.concatenate((all_train_input, all_test_input), axis=0))**2,axis=(0, 2, 3))
var_output=np.nanmean((np.concatenate((all_train_output, all_test_output), axis=0))**2,axis=(0, 2, 3))
print("mean and variance of all input data:")
print(mean_input,var_input)
print("mean and variance of all output data:")
print(mean_output,var_output)
#Scale the data so that they have variance of 1
all_train_input=all_train_input/np.sqrt(var_input[None, :, None, None])
all_train_output=all_train_output/np.sqrt(var_output[None, :, None, None])
all_test_input=all_test_input/np.sqrt(var_input[None, :, None, None])
all_test_output=all_test_output/np.sqrt(var_output[None, :, None, None])
#Have checked that after these operations, the data is scaled to be zero mean and variance 1.

for iensemble in np.arange(nensemble):
    run_model(var_input_names, var_output_names, save_fn_prefix, N_inp, N_out, iensemble, R2_all, corr_all)  
print('R2 from the best models in each run are:')
print(R2_all)
print('corr from the best models in each run are:')
print(corr_all)

mean and variance of all input data:
[3.2595452e-09] [3.62891008e-11]
mean and variance of all output data:
[-5.16228102e-04 -9.83592627e-05] [9.36516511e-05 1.01456128e-04]
Model has  1.124418  million params


  0%|          | 1/300 [00:08<42:20,  8.50s/it]

R2: 0.009642860031619849  corr:  0.10471265168434726  pval:  0.0


  1%|          | 2/300 [00:16<40:10,  8.09s/it]

R2: 0.07112478071079265  corr:  0.46183758598794933  pval:  0.0


...

R2: 0.7678711035007214  corr:  0.8776997536130805  pval:  0.0


 39%|███▉      | 118/300 [11:55<18:57,  6.25s/it]

R2: 0.7685982916097682  corr:  0.8772937034735929  pval:  0.0


100%|██████████| 300/300 [28:13<00:00,  5.64s/it]


R2 from the best models in each run are:
[0.76925564 0.76784907 0.7624617  0.76859775]
corr from the best models in each run are:
[0.87765357 0.87739345 0.87562212 0.87729587]


In [12]:
vi1 = 'div'

vo1 = 'ssh_cos'

save_fn_prefix  = 'any_{}_{}_smallbatch_cosonly_nobatchnorm'.format(vi1, vo1)
var_input_names = [vi1]
var_output_names = [vo1]

batch_size = 100

N_inp = len(var_input_names)
N_out = len(var_output_names)

lr0 = 0.005*10/batch_size #Roughly should scale inversely to batch_size

#Recording performance metrics on test data after eaching training cycle
R2_all = np.zeros(nensemble)
corr_all = np.zeros(nensemble)

nctrains, nctest = hf.load_data_from_nc_as_lists(root_dir)

all_train_input, all_train_output = preload_data_vortdiv(nctrains, Ntrain)
all_test_input, all_test_output = preload_data_vortdiv(nctest, Ntest)

#Normalize data
#Compute mean and variance for normalization
mean_input=np.nanmean(np.concatenate((all_train_input, all_test_input), axis=0),axis=(0, 2, 3))
mean_output=np.nanmean(np.concatenate((all_train_output, all_test_output), axis=0),axis=(0, 2, 3))
#Subtract the data with their means
all_train_input=all_train_input-mean_input[None, :, None, None]
all_train_output=all_train_output-mean_output[None, :, None, None]
all_test_input=all_test_input-mean_input[None, :, None, None]
all_test_output=all_test_output-mean_output[None, :, None, None]
#Compute the variances
var_input=np.nanmean((np.concatenate((all_train_input, all_test_input), axis=0))**2,axis=(0, 2, 3))
var_output=np.nanmean((np.concatenate((all_train_output, all_test_output), axis=0))**2,axis=(0, 2, 3))
print("mean and variance of all input data:")
print(mean_input,var_input)
print("mean and variance of all output data:")
print(mean_output,var_output)
#Scale the data so that they have variance of 1
all_train_input=all_train_input/np.sqrt(var_input[None, :, None, None])
all_train_output=all_train_output/np.sqrt(var_output[None, :, None, None])
all_test_input=all_test_input/np.sqrt(var_input[None, :, None, None])
all_test_output=all_test_output/np.sqrt(var_output[None, :, None, None])
#Have checked that after these operations, the data is scaled to be zero mean and variance 1.

for iensemble in np.arange(nensemble):
    run_model(var_input_names, var_output_names, save_fn_prefix, N_inp, N_out, iensemble, R2_all, corr_all)  
print('R2 from the best models in each run are:')
print(R2_all)
print('corr from the best models in each run are:')
print(corr_all)

mean and variance of all input data:
[3.2595452e-09] [3.62891008e-11]
mean and variance of all output data:
[-0.00051623] [9.36516511e-05]
Model has  1.124401  million params


  0%|          | 1/300 [00:05<29:38,  5.95s/it]

R2: -0.003531670180131119  corr:  0.0835755161125129  pval:  0.0


  1%|          | 2/300 [00:11<29:24,  5.92s/it]

R2: 0.001594867704651226  corr:  0.15529418134746084  pval:  0.0


...

R2: 0.6881343706629435  corr:  0.8325905295249125  pval:  0.0


 17%|█▋        | 52/300 [04:29<22:13,  5.38s/it]

R2: 0.6950110770180709  corr:  0.8376715986141466  pval:  0.0


100%|██████████| 300/300 [23:03<00:00,  4.61s/it]


R2 from the best models in each run are:
[0.70388961 0.69786856 0.70412668 0.69500771]
corr from the best models in each run are:
[0.84168713 0.83839298 0.84008428 0.83767132]
